**AST4310, Autumn 2021, Python version**

# Project 6c: Polarised Radiative Transfer


#### Header and imports

The cells below contain some code to label equations in Markdown and some recommended python imports to solve the exercises.

In [6]:
import numpy
from astropy import units
from scipy import special 

## 1. Background

Polarisation is a hidden property of radiation. In addition to intensity, observing the polarisation state of radiation allows for additional diagnostics of astrophysical media. The two main sources of polarisation in astrophysics are scattering polarisation and magnetic fields (via the Zeeman effect). In this project, you will explore simple solutions of polarised radiative transfer assuming only Zeeman effect. You will build on the work you did in Project 4 and 5, and explore the validity of commonly-used solutions and approximations. Throughout this project, you will work under the assumption of LTE ($S_\nu = B_\nu(T)$, and use of Saha-Boltzmann to obtain atomic populations).


### 1.1 Zeeman Effect

The Zeeman effect is the splitting of magnetic levels due to an external magnetic field. The strength and pattern of the splitting depend on the electronic configuration of the upper and lower levels. For any splitting to occur, the total angular momentum quantum number $J$ needs to be larger than zero. The splitting of a given level occurs for different values of the secondary total angular momentum quantum number $M_j$ (the projections of $J$ over an arbitrary z axis): $M_j = -J, \cdots, J$, so there are $2J + 1$ splits on a given level. Any transitions need to obey the electric dipole selection rule of $\Delta M_j = 0, \pm1$. The amount of splitting in energy is given by:

$$
\Delta E_B = \mu_B B (g^uM_j^u-g^lM_j^l),
$$
where $B\equiv\lVert\vec{B}\rVert$, $\mu_B$ is Bohr's magneton:

$$
\mu_B \equiv \frac{e\hbar}{2m_e},
$$
and $g^u$ and $g^l$ are the Landé factors for the upper and lower levels, for LS coupling given by:

$$
g = 1 + \frac{1}{2}\frac{J(J+1) + S(S+1) - L(L+1)}{J(J+1)}.
$$
For $J=0$, $g=1$.

In wavelength, the Zeeman splitting is given by:

$$
\lambda = \lambda_0 - \Delta\lambda_B (g^uM_j^u-g^lM_j^l),
$$
where $\lambda_0$ is the transition wavelength and 

$$
\Delta\lambda_B = \frac{e}{4\pi m_e c}\lambda^2 B.
$$

When $S=0$ or $g^u=g^l$, we have only three Zeeman components, a regime called normal Zeeman effect. These components correspond to the three scenarios of the selection rule $\Delta M_j = 0, \pm1$, and are called $\pi$ ($\Delta M_j=0$), $\sigma_r$ ($\Delta M_j=+1$), and $\sigma_b$ ($\Delta M_j=-1$). For other situations, we have the so-called anomalous Zeeman effect. For transitions with a complex splitting pattern, the effective Landé factor $\bar{g}$ is a useful concept. It measures the distance between $\lambda_0$ and "centre of gravity" of the $\sigma$ components, and is defined as:

\begin{eqnarray*}
\bar{g} &=& \frac{1}{2}(g_1+g_1) + \frac{1}{4}(g_1 - g_2)d\\
d &=& J_1(J_1+1)-J_2(J_2+1).
\end{eqnarray*}

In the anomalous Zeeman effect, not all components have the same strength. This strength is often called $S_q^{J^l J^u}$, is dependent on $q=\Delta M_j$, and $J$ for the upper and lower levels. The $S_q^{J^l J^u}$ are tabulated in [de la Cruz Rodriguez & van Noort (2017, Table 1, page 116)](https://ui.adsabs.harvard.edu/abs/2017SSRv..210..109D/abstract), and are coded in the function `zeeman_strength()` below. 

Each Zeeman component gives rise to one line profile. These are often called $\phi_-$, $\phi_0$, and $\phi_+$, respectively for $\Delta M_j=-1, 0, 1$, and can be written, for the normal Zeeman effect (assuming no macroscopic velocities):

\begin{eqnarray*}
\phi_0 &=& H\left(a, v\right)\\
\phi_\pm &=& H\left(a, v \mp \bar{g}\frac{\Delta\lambda_B}{\Delta\lambda_D} \right),
\end{eqnarray*}
Where $H(a, v)$ is the Voigt profile, and $\Delta\lambda_D$ the Doppler broadening. In the case of more than three components, one needs to sum over all the allowed transitions (for values of $M_j^u$ and $M_j^l$) and weigh the profiles by the Zeeman strength:

$$
\phi_q = \sum_{u, l} S_q^{J^l J^u} H\left(a, v + \left[g^uM_j^u-g^lM_j^l\right]\frac{\Delta\lambda_B}{\Delta\lambda_D} \right),
$$
where $q\equiv\Delta M_j=-1, 0, 1$.

Here are some of the expressions above coded into functions, for your convenience:



In [2]:
def g_LS(L, S, J):
    """
    Calculates Landé factor from LS coupling.
    """
    if J == 0:
        return 1
    else:
        return 1 + 0.5 * (J * (J + 1) + S * (S + 1) - L * (L + 1)) / (J * (J + 1))


def g_eff(g1, g2, J1, J2):
    """
    Calculates the effective Landé factor.
    """
    d = J1 * (J1 + 1) - J2 * (J2 + 1)
    return 0.5 * (g1 + g2) + 0.25 * (g1 - g2) * d


def zeeman_strength(J_l, J_u, M_l, M_u):
    """
    Calculates strengths of Zeeman components.
    """
    J, M = J_l, M_l
    if M_u - M_l == 1:
        if J_u - J_l == 1:
            strength = (3 * (J + M + 1) * (J + M + 2)) / (2 * (J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = (3 * (J - M) * (J + M + 1)) / (2 * J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J - M) * (J - M - 1)) / (2 * J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    elif M_u - M_l == 0:
        if J_u - J_l == 1:
            strength = (3 * (J - M + 1) * (J + M + 1)) / ((J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = 3 * M**2 / (J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J - M) * (J + M)) / (J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    elif M_u - M_l == -1:
        if J_u - J_l == 1:
            strength = (3 * (J - M + 1) * (J - M + 2)) / (2 * (J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = (3 * (J + M) * (J - M + 1)) / (2 * J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J + M) * (J + M - 1)) / (2 * J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    else:
        raise ValueError('Invalid transition, M_u - M_l != -1, 0, 1')
    return strength

### 1.2 Polarised Radiative Transfer Equation

For the polarised case, the radiative transfer equation (RTE) becomes a vector equation, where we no longer solve just for intensity $I_\nu$, but for the Stokes vector $\vec{I_\nu}$:

$$
\vec{I_\nu} = 
\begin{pmatrix}
I_\nu \\
Q_\nu \\
U_\nu \\
V_\nu 
\end{pmatrix}.
$$

The RTE then becomes:
$$
\frac{\mathrm{d}\vec{I_\nu}}{\mathrm{d}s} = \vec{j_\nu} - \mathbb{A}_\nu\vec{I_\nu},
$$

where $\mathbb{A}_\nu$ is the absorption matrix and $\vec{j_\nu}$ the emissivity vector. In the case where there is no continuum emission of polarised radiation (true in most stars), we can simplify $\mathbb{A}_\nu$:

\begin{eqnarray*}
\mathbb{A}_\nu &=& \alpha_\nu^c \mathbb{1} + \mathbb{A}_\nu^l \\
\mathbb{A}_\nu &=& \alpha_\nu^c \mathbb{1} + \alpha_\nu^l \Phi_\nu,
\end{eqnarray*}

where $\alpha_\nu^c$ is the continuous extinction coefficient and $\alpha_\nu^l$ is the line extinction coefficient  ***without the line profile part*** $H(a, v)$. The radiative transfer equation can then be further simplified by dividing by $\alpha_\nu^c$ and using $\eta_\nu\equiv\alpha_\nu^l / \alpha_\nu^c$:


$$
\frac{\mathrm{d}}{\mathrm{d}\tau_\nu^c}
\begin{pmatrix}
I_\nu \\
Q_\nu \\
U_\nu \\
V_\nu 
\end{pmatrix} = 
\vec{S_\nu} - (\mathbb{1} + \eta_\nu \Phi_\nu) 
\begin{pmatrix}
I_\nu \\
Q_\nu \\
U_\nu \\
V_\nu 
\end{pmatrix}.
$$





The line profile information goes into the matrix $\Phi$, which in general can be written as:

$$
\Phi = 
\begin{pmatrix}
\phi_I & \phi_Q & \phi_U & \phi_V \\
\phi_Q & \phi_I & \psi_V & -\psi_U \\
\phi_U & -\psi_V & \phi_I & \psi_Q \\
\phi_V & \psi_U & -\psi_Q & \phi_I 
\end{pmatrix}. 
$$

This matrix has 7 independent terms, and can be divided into three parts:


$$
\Phi  = 
\begin{pmatrix}
\phi_I & 0 & 0 & 0 \\
0 & \phi_I & 0 & 0 \\
0 & 0 & \phi_I & 0 \\
0 & 0 & 0 & \phi_I 
\end{pmatrix} +
%
\begin{pmatrix}
0 & \phi_Q & \phi_U & \phi_V \\
\phi_Q & 0 & 0 & 0 \\
\phi_U & 0 & 0 & 0 \\
\phi_V & 0 & 0 & 0 
\end{pmatrix} +
%
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & \psi_V & -\psi_U \\
0 & -\psi_V & 0 & \psi_Q \\
0 & \psi_U & -\psi_Q & 0 
\end{pmatrix}.
$$

The first term represents absorption of energy, the second term represents dichroism, and the last term represents magneto-optical effects.

Under Zeeman effect, for a magnetic field with inclination angle $\gamma$ and azimuthal angle $\chi$, the $\phi$ profiles are given by:

\begin{eqnarray*}
\phi_I &=& \phi_\Delta\sin^2\gamma + \frac{1}{2}(\phi_+ + \phi_-) \\
\phi_Q &=& \phi_\Delta\sin^2\gamma \cos 2\chi \\
\phi_U &=& \phi_\Delta\sin^2\gamma \sin 2\chi \\
\phi_V &=& \frac{1}{2}(\phi_+ - \phi_-) \cos \gamma \\
\phi_\Delta &=& \frac{1}{2} \left[ \phi_0 + \frac{1}{2}(\phi_+ + \phi_-)\right],
\end{eqnarray*}

where $\phi_-$, $\phi_0$, and $\phi_+$ are the line profiles for the Zeeman components defined in section 1.1. The $\psi$ profiles, for the magneto-optical effects, are defined similarly:

\begin{eqnarray*}
\psi_I &=& \psi_\Delta\sin^2\gamma + \frac{1}{2}(\psi_+ + \psi_-) \\
\psi_Q &=& \psi_\Delta\sin^2\gamma \cos 2\chi \\
\psi_U &=& \psi_\Delta\sin^2\gamma \sin 2\chi \\
\psi_V &=& \frac{1}{2}(\psi_+ - \psi_-) \cos \gamma \\
\psi_\Delta &=& \frac{1}{2} \left[ \psi_0 + \frac{1}{2}(\psi_+ + \psi_-)\right],
\end{eqnarray*}

where the $\psi_-$, $\psi_0$, and $\psi_+$ are defined very similarly to the $\phi$ counterparts. E.g. in the case of normal Zeeman effect:

\begin{eqnarray*}
\psi_0 &=& L\left(a, v + \frac{\Delta\lambda_\mathrm{los}}{\Delta\lambda_D}\right)\\
\psi_\pm &=& L\left(a, v + \frac{\Delta\lambda_\mathrm{los}}{\Delta\lambda_D} \mp \bar{g}\frac{\Delta\lambda_B}{\Delta\lambda_D} \right).
\end{eqnarray*}

The only change is from the Voigt profile $H(a, v)$ to the Dispersion function (sometimes called Faraday profile) $L(a, v)$. The two functions are related, as they are respectively the real and imaginary parts of the [Fadeeva function](https://en.wikipedia.org/wiki/Faddeeva_function):

$$
\mathcal{H}(a, v) = H(a, v) + iL(a, v).
$$

In python, both functions can be computed via `scipy.special`:

In [3]:
def voigt(gamma, x):
    """
    Calculates the Voigt function.
    """
    z = (x + 1j * gamma)
    return special.wofz(z).real / numpy.sqrt(numpy.pi)


def faraday(gamma, x):
    """
    Calculates the Faraday dispersion function.
    """
    z = (x + 1j * gamma)
    return special.wofz(z).imag / numpy.sqrt(numpy.pi)

### 1.3 The Unno-Rachkovsky Solution

A simplified solution for the polarised RTE is the Unno-Rachkovsky solution. In this solution, one uses the simplifying assumption that the source function (in LTE, $B_\nu$) varies linearly with the continuum optical depth:

$$
S_\nu = S_0 + S_1\tau_c.
$$

An atmosphere that fulfils the above is called a *Milne-Eddington* atmosphere. Moreover, to greatly simplify the polarised RTE, this solution assumes that everything that affects the absorption matrix $\mathbb{A}_\nu$ is constant with height:

* Magnetic field vector: $B$, $\gamma$, $\chi$
* Ratio of extinctions: $\eta_\nu$
* Broadening of the line: $a$ and $\Delta\lambda_D$.

When the above are fulfilled, there is an exact solution to the polarised RTE, and it can be written explicitly (dropping the frequency indices) as:

\begin{eqnarray*}
I &=& S_0 + \Delta^{-1}\left[k_I(k_I^2+ f_Q^2+f_U^2+f_V^2)\right]S_1 \\
Q &=& -\Delta^{-1} \left[k_I^2 k_Q + k_I(k_V f_U- k_U f_V) + f_Q\Pi\right] S_1 \\
U &=& -\Delta^{-1} \left[k_I^2 k_U + k_I(k_Q f_V- k_V f_Q) + f_U\Pi\right] S_1 \\
V &=& -\Delta^{-1} \left[k_I^2 k_V + f_V\Pi\right] S_1,
\end{eqnarray*}
where 
\begin{eqnarray*}
\Delta & = & k_I^2(k_I^2 - k_Q^2-k_U^2-k_V^2 + f_Q^2+f_U^2+f_V^2) + \Pi^2 \\
\Pi &=& k_Q f_Q + k_U f_U + k_V f_V,
\end{eqnarray*}

and
\begin{eqnarray*}
k_I  =  1 + \eta_\nu \phi_I, \;\; k_Q&=&\eta_\nu \phi_Q, \;\;   k_U = \eta_\nu \phi_U, \;\;   k_V = \eta_\nu \phi_V\\
  f_Q &=& \eta_\nu \psi_Q, \;\;  f_U = \eta_\nu \psi_U, \;\; f_V = \eta_\nu \psi_V.
\end{eqnarray*}

Below you can find an implementation of the Unno-Rachkovsky solution in python. The arguments of this function are the free parameters for the above: $\gamma$, $\chi$, $\eta_\nu$, $a$, and $\Delta\lambda_B / \Delta\lambda_D$. While this last ratio can just be given as a ratio, for detailed calculations one needs to compute explicitly $\Delta\lambda_B$ for a given value of the magnetic field strength. This function only works for the normal Zeeman effect, but modification for arbitrary Zeeman patterns is just a matter of changing $\phi_{0}$, $\phi_\pm$, $\psi_{0}$, and $\psi_\pm$ to the appropriate expressions summing over components.

In [4]:
def unno_rachkovsky(u, s0=1, s1=5, eta=20, a=0.05, g_eff=1, 
                    delta_ratio=1.5, gamma=numpy.pi/3, chi=0):
    """
    Calculates Stokes vector using Unno-Rachkovsky solution, for a given 
    source function S = s0 + s1 * tau.
    
    Parameters
    ----------
    u : 1-D array
        Dimensionless wavelength in Doppler width units
    s0, s1: scalar (astropy intensity units)
        Constants in the definition of source function.
    eta : scalar
        Ratio of line to continuum extinction, alpha_l / alpha_c.
    a: scalar
        Broadening of profile
    u: 1-D array
        Normalised wavelength scale. 
    g_eff: scalar
        Effective Lande factor.
    delta_ratio: scalar
        Ratio of Zeeman broadening to Doppler broadening.
    gamma: scalar
        Inclination angle of magnetic field
    chi: scalar
        Azimuth angle of magnetic field
    """
    phi_0 = voigt(a, u)
    phi_r = voigt(a, u + g_eff * delta_ratio) 
    phi_b = voigt(a, u - g_eff * delta_ratio) 
    psi_0 = faraday(a, u)
    psi_r = faraday(a, u + g_eff * delta_ratio) 
    psi_b = faraday(a, u - g_eff * delta_ratio)
    
    phi_delta = 0.5 * (phi_0 - 0.5 * (phi_b + phi_r))
    phi_I = phi_delta * numpy.sin(gamma)**2 + 0.5 * (phi_b + phi_r)
    phi_Q = phi_delta * numpy.sin(gamma)**2 * numpy.cos(2 * chi)
    phi_U = phi_delta * numpy.sin(gamma)**2 * numpy.sin(2 * chi)
    phi_V = 0.5 * (phi_b - phi_r) * numpy.cos(gamma)
    
    psi_delta = 0.5 * (psi_0 - 0.5 * (psi_b + psi_r))
    psi_Q = psi_delta * numpy.sin(gamma)**2 * numpy.cos(2 * chi)
    psi_U = psi_delta * numpy.sin(gamma)**2 * numpy.sin(2 * chi)
    psi_V = 0.5 * (psi_b - psi_r) * numpy.cos(gamma)
    
    kI = 1 + eta * phi_I
    kQ = eta * phi_Q
    kU = eta * phi_U
    kV = eta * phi_V
    fQ = eta * psi_Q
    fU = eta * psi_U
    fV = eta * psi_V

    delta = (kI**4 + kI**2 * (fQ**2 + fU**2 + fV**2 - kQ**2 - kU**2 - kV**2) - 
             (kQ * fQ + kU * fU + kV * fV)**2)
    I = s0 + s1 / delta * kI * (kI**2 + fQ**2 + fU**2 + fV**2)
    Q = -s1 / delta * (kI**2 * kQ - kI * (kU * fV - kV * fU) + fQ * (kQ * fQ + kU * fU + kV * fV))
    U = -s1 / delta * (kI**2 * kU - kI * (kV * fQ - kQ * fV) + fU * (kQ * fQ + kU * fU + kV * fV))
    V = -s1 / delta * (kI**2 * kV + fV * (kQ * fQ + kU * fU + kV * fV))
    return I, Q, U, V

---

### Exercise 1: Zeeman effect and Polarisation [32 points]

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* Assume that the terms $^5P$ and $^5D$ can define respectively the upper and lower level of a bound-bound transition. How many different levels does each term have, and how many permited transitions are possible with Zeeman splitting? Remember that a level is set by a fixed S, L, and J. The quantum mechanical selection rules for electric dipole transitions are $\Delta M_j=0, \pm 1$ and $\Delta J = 0, \pm 1$ (as long as $J^u$ and $J^l$ are not both zero). 
* Calculate the effective Landé factor $\bar{g}$ for the $^5 P_1 - {}^5D_1$ transition. You will need this value in exercise 3 for the Ti I 2.222 $\mu$m line.
* Using the definitions of the Stokes profiles, show that we always have $I^2 \ge Q^2 + U^2 + V^2$.
* What happens to RTE solutions when you change $\chi$ by 180º? And when you change $\gamma$ by 180º?

</div>

### Exercise 2: The Unno-Rachkovsky solution [21 points]

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* Chose some appropriate values for a Unno-Rachkovsky solution. Plot the Stokes profiles for cases with $\chi=0$, for several values of $0 \le \gamma \le \pi$. And then, for $\gamma=\pi/4$ and $0 \le \chi \le 2\pi$. Plot as line profiles (4 panels), and as spectrogram, 4 images of wavelength (x axis) and angle (y axis), I, Q, U, V side by side. What can you learn?
* Consider a Unno-Rachkovsky solution with parameters of $S_0=1$, $S_1=5$, $\eta=5$, $a=0.05$, $\bar{g}=2.5$, $\Delta\lambda_B / \Delta\lambda_D=1.5$, $\gamma=\pi/4$, $\chi=0$. In this case, for intensity both $\pi$ and $\sigma$ components are fully split. Only two of these parameters influence the relative strength of the three Zeeman components. Which ones, and why? Plot a case where all three components have approximately the same line depth in intensity.
* How important are magneto-optical effects? For one or more combinations of parameters, plot the Stokes profiles with and without magneto-optical effects. Discuss.
</div>

### Exercise 3: Milne-Eddington lines from the FALC model [42 points]

In this exercise, you are going to test the validity of the Unno-Rachkovsky solution and the Milne-Eddington approximation. Is it really a good fit for lines in stellar atmospheres? You will also test when the weak field approximation you used in Project 3 can be valid. For this, you will compute profiles of the same Fe I line that was observed for Project 3, the Fe I line at 617.3 nm. In addition, you will also use a neutral titanium line observed in the infrared. The line properties are as follows:

|                  | Fe I 617 | Ti I 2221 |
|------------------|-----------:|-----------:|
| Air wavelength (nm)  | 617.333         | 2221.122  |
| Lower level      |  4s ${}^5P_{1}$ | 4s $^5P_1$ |
| Upper level      |  4p ${}^5D_{0}$ | 4p  ${}^5D_1$|
| $\chi_{1,l}$ (aJ)| 0.35611  | 0.27774      |
| $\chi_{1,u}$ (aJ)| 0.67780 | 0.36715 |
| $\chi_{1,\infty}$ (aJ) | 1.26610 | 1.09274 | 
|   $g_l$          | 3     | 3      |
|   $g_u$          | 1     | 3      |
| $f_{lu}$         | 4.39e-4 | 5.55e-3  |
| $\Delta \overline{r^2}$ |   11.65  | 5.89   |
| Element abundance | 3.162e-5   | 8.913e-8     |
| Atomic mass (u) | 55.845   | 47.867 |

Here $g_l$ and $g_u$ denote the statistical weights, not the Landé factors. For some of the questions below you will need to compute the disk-centre intensity for those two lines with the FALC model, in a similar fashion to what you did for the Na I D$_1$ line in Project 5. Feel free to use the same routines and code you used earlier (e.g. to compute populations from Saha-Boltzmann and line extinction). Here are a few simplifications to make that task easier. Fe I and Ti I are larger atoms than Na I. They are not hydrogenic, have more valence electrons, and a lot more levels. You can use the same Unsöld expression for van der Waals broadening (with $P_g$ in cgs and $T$ in K):

\begin{equation*}
     \log \gamma_\mathrm{vdW} \approx 6.33
                     + 0.4 \log \Delta \overline{r^2}
                     + \log P_g - 0.7 \log T   \;\;\;\left[\log\left(\mathrm{s}^{-1}\right)\right],
    \label{eq:vanderWaals}
\end{equation*}

but do not calculate $\Delta \overline{r^2}$ as in Project 5 (formula there is for hydrogenic species). Instead, the appropriate $\Delta \overline{r^2}$ for each line can be found in the table above.

A couple of simplified model atoms for Fe I and Ti I are also provided (`FeI_atom.txt` and `TiI_atom.txt`) to make it easier to use your previous code (keep in mind that, unlike Na I D$_1$, these are not resonance lines - *the lower level of the lines is not the lowest level in the atom file*). These atom files have only the transition levels, and the ground levels of neutral and ionised species, and therefore are not suitable to compute the partition functions. To compute the partition functions $U_r$, it is recommended you use the recipe from Gray (2005) in the code below, and do not rely on building model atoms and calculating $U_r$ as in previous projects. The `temp_tab` array below gives the tabulated temperature, and `log_u_XX` gives $\log_{10}U_r(T)$ for a given species. To get the partition functions at any temperature, you need only to interpolate these tables. 

In [7]:
# Partition functions from Gray
theta = numpy.arange(0.2, 2.2, 0.2)
temp_tab = (5040. * units.K / theta)
log_u_FeI = numpy.array([3.760, 2.049, 1.664, 1.519, 1.446, 1.402, 1.372, 1.350, 1.332, 1.317])
log_u_FeII = numpy.array([2.307, 1.881, 1.749, 1.682, 1.638, 1.604, 1.575, 1.549, 1.525, 1.504])
log_u_TiI = numpy.array([4.159, 2.333, 1.818, 1.596, 1.480, 1.411, 1.367, 1.337, 1.316, 1.300])
log_u_TiII = numpy.array([1.524, 1.435, 1.374, 1.338, 1.315, 1.300, 1.289, 1.280, 1.272, 1.265])

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">
    
* Calculate the effective Landé factor for the Fe I 617.3 nm line. Does this line exhibit normal or anomalous Zeeman effect?
    
* The weak field approximation is valid when $\bar{g}\Delta\lambda_B/\Delta\lambda_D \ll 1$. In Project 3, you worked with observations of the 617.3 nm line, and inferred a maximum magnetic field strength of about 0.2 T. Is the weak field approximation valid in this case? Justify.
    
* Is the Unno-Rachkovsky solution appropriate for the Fe I 617.3 nm line with the FALC model? Test this by comparing the disk-centre intensity computed in LTE as you did in Project 5, with the intensity (B=0) given by Unno-Rachkovsky. To obtain the mean quantities for the Unno-Rachkovsky solution, you will need to find the formation range of the spectral line and in that region do a linear fit of $B_\lambda(T)$ vs $\tau_{500}$ to extract $S_0$ and $S_1$ (the other quantities such as $\Delta\lambda_D$ can be taken as the average in the region).

* Consider the Unno-Rachkovsky solution for Fe I 617.3 nm with parameters found in the previous question, assume a vertical magnetic field. What value of B would you need to see the split between $\sigma_r$ and $\sigma_b$ in the intensity profile?

* Repeat the previous question but for the Ti I 2.221 $\mu$m line, accounting for all Zeeman components. (You will need to extract the mean quantities for this line using the same process as before.) What value of B would you need to see the split among the individual $\sigma_b$ components (not just between $\sigma_r$ and $\sigma_b$)?
    

*Hints:* Neither the Fe I or Ti I line should show a central reversal. To obtain $S_0$ and $S_1$ for the Unno-Rachkovsky solution, you need to do a linear fit to the source function ($B_\lambda$) over the range of heights where a line is formed. This range can be found in a similar way to what you did in Project 5 where you computed the height of formation of radiation at different wavelengths. You can obtain the range of formation heights of a line by computing the height of formation for two wavelengths, one in the far wings and another at the very line core (e.g. use equation (4) from Project 5). In the case of the Ti I line, the range of formation between wings and core is much narrower, as the line extinction is not much larger than the continuum extinction. For this line, you may have to adjust the range of formation heights to obtain a similar intensity in Unno-Rachkovsky as in the formal solver of Project 5 (in this case, it is ok to adjust the range until the two profiles are roughly similar).

</div>